### Response Completeness Evaluator

### Getting Started

This sample demonstrates how to use Response Completeness Evaluator
Before running the sample:
```bash
pip install azure-ai-projects azure-identity azure-ai-evaluation
```
Set these environment variables with your own values:
1) **PROJECT_CONNECTION_STRING** - The project connection string, as found in the overview page of your Azure AI Foundry project.
2) **MODEL_DEPLOYMENT_NAME** - The deployment name of the AI model, as found under the "Name" column in the "Models + endpoints" tab in your Azure AI Foundry project.
3) **AZURE_OPENAI_ENDPOINT** - Azure Open AI Endpoint to be used for evaluation.
4) **AZURE_OPENAI_API_KEY** - Azure Open AI Key to be used for evaluation.
5) **AZURE_OPENAI_API_VERSION** - Azure Open AI Api version to be used for evaluation.
6) **AZURE_SUBSCRIPTION_ID** - Azure Subscription Id of Azure AI Project
7) **PROJECT_NAME** - Azure AI Project Name
8) **RESOURCE_GROUP_NAME** - Azure AI Project Resource Group Name

The Response Completeness evaluator assesses the quality of an agent response by examining how well it aligns with the provided ground truth. The evaluation is based on the following scoring system:

<pre>
Score 1: Fully incomplete: The response misses all necessary and relevant information compared to the ground truth.
Score 2: Barely complete: The response contains only a small percentage of the necessary information.
Score 3: Moderately complete: The response includes about half of the necessary information.
Score 4: Mostly complete: The response contains most of the necessary information, with only minor omissions.
Score 5: Fully complete: The response perfectly matches all necessary and relevant information from the ground truth.
</pre>

The evaluation requires the following inputs:

Response: The response to be evaluated. (string)
Ground Truth: The correct and complete information against which the response is compared. (string)

The evaluator uses these inputs to determine the completeness score, ensuring that the response meaningfully addresses the query while adhering to the provided definitions and data.

### Initialize Completeness Evaluator


In [ ]:
from azure.ai.evaluation import ResponseCompletenessEvaluator , AzureOpenAIModelConfiguration
from pprint import pprint
import os

model_config = AzureOpenAIModelConfiguration(
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    api_key=os.environ["AZURE_OPENAI_API_KEY"],
    api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    azure_deployment=os.environ["MODEL_DEPLOYMENT_NAME"],
)


In [ ]:
from azure.ai.evaluation import ResponseCompletenessEvaluator , AzureOpenAIModelConfiguration
from pprint import pprint

response_completeness_evaluator = ResponseCompletenessEvaluator(model_config=model_config)

### Samples

#### Evaluating for a ground_truth and prediction

In [ ]:
result = response_completeness_evaluator(
    response="The capital of Japan",
    ground_truth="The capital of Japan is Tokyo."
)
result

In [ ]:
result = response_completeness_evaluator(
    response="The capital of Japan is Tokyo.",
    ground_truth="The capital of Japan is Tokyo."
)
result

#### Evaluate with a reasoning model

In [ ]:
from azure.ai.evaluation import ResponseCompletenessEvaluator , AzureOpenAIModelConfiguration
from pprint import pprint

# set is_reasoning_model to True in case the model is a reasoning model (ex: o3-mini, o1-preview)
response_completeness_evaluator = ResponseCompletenessEvaluator(model_config=model_config,
                                                                is_reasoning_model=True)

result = response_completeness_evaluator(
    response="The capital of Japan is Tokyo.",
    ground_truth="The capital of Japan is Tokyo."
)
result

# Batch run for response completeness

In [ ]:
import json
import pandas as pd

data = [
    {
        "response": "The temperature of Seattle now is 70 degrees. Based on the temperature, having an outdoor office party is recommended.",
        "ground_truth": "The temperature of Seattle now is 50 degrees. It will be recommended to bring a jacket in the evening.",
    },
    {
        "response": "The email draft \"Project Plan\" is attached. Please review and provide feedback.",
        "ground_truth": "The email draft \"Project Plan\" is attached. Please review and provide feedback by EOD.",
    },
    {
        "response": "Based on the retrieved documents, the shareholder meeting discussed the operational efficiency of the company and financing options.",
        "ground_truth": "The shareholder meeting discussed the compensation package of the company CEO.",
    }
]

file_path = "response_completeness_data.jsonl"

pd.DataFrame(data).to_json(
    file_path, orient="records", lines=True
)

from azure.ai.evaluation import evaluate

azure_ai_project={
        "subscription_id": os.environ["AZURE_SUBSCRIPTION_ID"],
        "project_name": os.environ["PROJECT_NAME"],
        "resource_group_name": os.environ["RESOURCE_GROUP_NAME"],
    }

response = evaluate(
    data=file_path,
    evaluators={
        "response_completeness": response_completeness_evaluator,
    },
    azure_ai_project=azure_ai_project,
)

pprint(f'AI Foundry URL: {response.get("studio_url")}')